In [ ]:
import pandas as pd
import os
import numpy as np

pd.set_option('display.max_columns', None)

# --- 1. CARREGAMENTO DAS TRÊS FONTES DE DADOS ---
caminho_ofertas = '../../../planilhas/limpo/modulo_3/limpar_e_padronizar_colunas/ofertas_limpo.csv'
caminho_inscricoes_geral = '../../../planilhas/limpo/modulo_3/limpar_e_padronizar_colunas/inscritos_limpo.csv'
caminho_candidatos_validado= "../../../planilhas/processado/modulo_3/funil_e_candidatos_unicos/candidatos_unicos_processado.csv"

dfo = pd.read_csv(caminho_ofertas)
dfi_geral = pd.read_csv(caminho_inscricoes_geral)
dfi_candidatos_unicos= pd.read_csv(caminho_candidatos_validado)





'''
APENAS nome das 10 areas gerais CINE
'''






# --- 2. CÁLCULO DE CADA ETAPA DO FUNIL ---
chave_agrupamento = ['ano', 'semestre', 'nome_cine_area_geral', "uf_local_oferta"]
ordem_sort = ['ano', 'semestre', 'id_estudante', 'situacao_fies']
subset_drop = ['ano', 'semestre', 'id_estudante']





# --- Etapas de Ofertas ---
df_vagas_ofertadas = dfo.groupby(chave_agrupamento, as_index=False)['vagas_fies'].sum()

df_vagas_ocupadas = dfo.groupby(chave_agrupamento, as_index=False)['vagas_ocupadas'].sum()



# --- Etapas de Inscrições (Gerais) ---
df_inscritos_geral = (dfi_geral
                      .groupby(chave_agrupamento, as_index=False)
                      .size()
                      .rename(columns={'size': 'Inscritos_Geral'}))





# --- Etapas de Candidatos Únicos (Gerais)


df_candidatos_unicos_geral = (dfi_candidatos_unicos
                              .groupby(chave_agrupamento, as_index=False)
                              .size()
                              .rename(columns={'size': 'Candidatos_Unicos_Geral'}))



# --- Etapas "CONCORRENDO por nota enem > nota corte grupo" de múltiplas fontes ---

# garantir que sao numeros media do enem e a nota de corte e padrao brasileiro
for df_temp in [dfi_geral, dfi_candidatos_unicos]:

    df_temp['media_enem'] = (pd
                                  .to_numeric(df_temp['media_enem']
                                              .astype(str)
                                              .str
                                              .replace(',', '.'), errors='coerce'))
    

    df_temp['nota_corte_gp'] = (pd
                                               .to_numeric(df_temp['nota_corte_gp']
                                                           .astype(str)
                                                           .str
                                                           .replace(',', '.'), errors='coerce'))




# --- Etapa: candidatos unicos com nt suficiente (Candidatos Únicos)-

FILTRO_TAL= dfi_candidatos_unicos['media_enem'] >= dfi_candidatos_unicos['nota_corte_gp']

df_base_candidatos_unicos_nota_suficiente = dfi_candidatos_unicos[FILTRO_TAL].copy()



df_candidatos_unicos_nota_suficiente = (df_base_candidatos_unicos_nota_suficiente
                        .groupby(chave_agrupamento, as_index=False)
                        .size()
                        .rename(columns={'size': 'candidatos_unicos_com_nota_suficiente'}))






# --- Etapa: Inscritos  com Nota Suficiente ---

FILTRO_TAL= dfi_geral['media_enem'] >= dfi_geral['nota_corte_gp']

df_inscritos_com_nota_base = dfi_geral[FILTRO_TAL].copy()

df_inscritos_com_nota = (df_inscritos_com_nota_base
                         .groupby(chave_agrupamento, as_index=False)
                         .size()
                         .rename(columns={'size': 'inscritos_com_nota_suficiente'}))




# --- 3. JUNÇÃO FINAL ---

df_final = df_vagas_ofertadas


dfs_para_juntar = [

#df_vagas_ofertadas, pq df_final o primeiro ja eh df vagas ofertas
df_inscritos_geral,
df_inscritos_com_nota,
df_candidatos_unicos_geral,
df_candidatos_unicos_nota_suficiente,
df_vagas_ocupadas,
]


for df_etapa in dfs_para_juntar:
    df_final = pd.merge(df_final, df_etapa, on=chave_agrupamento, how='outer')

df_final = df_final.fillna(0)

df_final

<ipython-input-1-cff8679b1004>:13: DtypeWarning: Columns (31,33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  dfo = pd.read_csv(caminho_ofertas)
<ipython-input-1-cff8679b1004>:14: DtypeWarning: Columns (25,37,38,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  dfi_geral = pd.read_csv(caminho_inscricoes_geral)
<ipython-input-1-cff8679b1004>:15: DtypeWarning: Columns (37,38,49) have mixed types. Specify dtype option on import or set low_memory=False.
  dfi_candidatos_unicos= pd.read_csv(caminho_candidatos_validado)


,ano,semestre,nome_cine_area_geral,uf_local_oferta,vagas_fies,Inscritos_Geral,inscritos_com_nota_suficiente,Candidatos_Unicos_Geral,candidatos_unicos_com_nota_suficiente,vagas_ocupadas
0,2019.0,1.0,"Agricultura, silvicultura, pesca e veterinária",AL,220.0,146.0,100.0,102.0,72.0,39.0
1,2019.0,1.0,"Agricultura, silvicultura, pesca e veterinária",AM,216.0,712.0,203.0,310.0,84.0,51.0
2,2019.0,1.0,"Agricultura, silvicultura, pesca e veterinária",BA,1898.0,1432.0,618.0,866.0,404.0,274.0
3,2019.0,1.0,"Agricultura, silvicultura, pesca e veterinária",CE,399.0,1168.0,361.0,726.0,236.0,147.0
4,2019.0,1.0,"Agricultura, silvicultura, pesca e veterinária",DF,267.0,627.0,91.0,350.0,67.0,39.0
...,...,...,...,...,...,...,...,...,...,...
1505,2021.0,2.0,Serviços,RS,446.0,36.0,32.0,26.0,24.0,5.0
1506,2021.0,2.0,Serviços,SC,490.0,13.0,13.0,9.0,9.0,0.0
1507,2021.0,2.0,Serviços,SE,242.0,26.0,12.0,17.0,9.0,2.0
1508,2021.0,2.0,Serviços,SP,5556.0,288.0,278.0,144.0,138.0,28.0


In [ ]:
df_final.to_csv('../../../planilhas/processado/modulo_3/funil_e_candidatos_unicos/funil_por_uf.csv', index=False)